# Change detection using M3C2 algorithm
With the help of the change detection algorithm M3C2 we can remove non-static objects from the point cloud. Resulting in a point cloud with only static objects, that we name obstacles. The provided example tile in the folder `../datasets/pointclouds/sidewalk/m3c2/` is generated from the M3C2 plugin inside CloudCompare. With the M3C2 implementation of [py4dgeo](https://github.com/ssciwr/py4dgeo) we can do almost the same. This notebook presents how the M3C2 algorithm can be run using the py4dgeo package.

In [ ]:
# Add project src to path.
import set_path

import numpy as np
import time
import os
import py4dgeo
import laspy

from upc_sw.las_utils_extra import write_las, read_las

In [ ]:
sidewalk_filter_run1 = '../datasets/pointclouds/sidewalk/run1/processed_2386_9702.laz'
sidewalk_filter_run2 = '../datasets/pointclouds/sidewalk/run2/processed_2386_9702.laz'
save_path = '../datasets/pointclouds/m3c2/'

Load two datasets that cover the same scene at two different points in time.

In [ ]:
epoch1, epoch2 = py4dgeo.read_from_las(
    py4dgeo.find_file(sidewalk_filter_run1),
    py4dgeo.find_file(sidewalk_filter_run2)
)

The distance analysis is executed on a number of points of interest called *core points*. This could be e.g. the entire reference point cloud, a downsampled version of it, an equistant grid etc. Here, we choose the entire point cloud:

In [ ]:
corepoints = epoch1.cloud

Next, we instantiate the algorithm class and run the distance calculation:

In [ ]:
"""
We mapped the CloudCompare variables from m3c2_params.txt with py4dgeo
See the mapping of "CloudCompare -> py4dgeo" here: https://github.com/ssciwr/py4dgeo/issues/84
We have to divide the values of two params by 2: https://github.com/ssciwr/py4dgeo/issues/128
"""
normal_scale = 0.70 / 2
search_scale = 0.70 / 2
max_distance = 3.55

start_time = time.time()

m3c2 = py4dgeo.M3C2(
    epochs=(epoch1, epoch2),
    corepoints=corepoints,
    cyl_radii=(search_scale,),
    normal_radii=(normal_scale,),
    max_distance=max_distance
)

distances, uncertainties = m3c2.run()

print("--- %s seconds ---" % (time.time() - start_time))

The calculated distances are saved to a new scalar field in the point cloud.

In [ ]:
points, _ = read_las(sidewalk_filter_run1)

pathname = os.path.join(save_path, os.path.basename(sidewalk_filter_run1))
write_las_m3c2(points, pathname, distances)